In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests

# 设置 ChromeDriver 路径
driver_path = '/path/to/chromedriver'  # 更改为你实际的 ChromeDriver 路径
url = 'https://www.etit.kit.edu/vertiefungsrichtungen_master.php'

# 使用 Selenium 启动浏览器并访问页面
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)

# 等待页面加载（可以调整等待时间）
time.sleep(3)

# 点击其中一个链接进入新页面
link = driver.find_element(By.LINK_TEXT, "Vertiefungsrichtung 2")  # 选择具体链接的文本
link.click()

# 等待页面加载
time.sleep(3)

# 获取页面源代码
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 查找页面中的所有 PDF 链接
pdf_links = soup.find_all('a', href=True)
pdf_urls = [link['href'] for link in pdf_links if link['href'].endswith('.pdf')]

# 下载每个 PDF 文件
for pdf_url in pdf_urls:
    full_url = f"https://www.etit.kit.edu{pdf_url}" if pdf_url.startswith('/') else pdf_url
    print(f"Downloading {full_url}")
    response = requests.get(full_url)
    
    # 设定保存路径和文件名
    pdf_filename = full_url.split('/')[-1]
    with open(pdf_filename, 'wb') as file:
        file.write(response.content)

# 关闭浏览器
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import os

# Set up the Chrome WebDriver path
driver_path = '/path/to/chromedriver'  # Change this to your actual ChromeDriver path
url = 'https://www.etit.kit.edu/vertiefungsrichtungen_master.php'  # Main page

# Create a folder to save PDFs
download_folder = 'downloaded_pdfs'
os.makedirs(download_folder, exist_ok=True)

# Set up the Selenium driver
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)

# Wait for the page to load
time.sleep(3)

# Find all links to direction pages
direction_links = driver.find_elements(By.PARTIAL_LINK_TEXT, 'vertiefungsrichtung_')  # Assumes all direction pages contain this text

# Loop through each direction page
for link in direction_links:
    # Get the href attribute and visit the direction page
    direction_url = link.get_attribute('href')
    print(f"Accessing: {direction_url}")
    driver.get(direction_url)
    
    # Wait for the page to load
    time.sleep(3)

    # Find all PDF links on the current page
    pdf_links = driver.find_elements(By.PARTIAL_LINK_TEXT, ".pdf")  # Find links containing ".pdf"

    # Download each PDF
    for pdf_link in pdf_links:
        pdf_url = pdf_link.get_attribute('href')
        print(f"Downloading {pdf_url}")

        # Download the PDF file using requests
        response = requests.get(pdf_url)
        pdf_filename = os.path.join(download_folder, pdf_url.split('/')[-1])  # Create a file path to save the PDF
        
        # Save the PDF to the designated folder
        with open(pdf_filename, 'wb') as file:
            file.write(response.content)

# Close the browser after the operation is complete
driver.quit()


In [3]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import shutil


# 设置下载文件的主文件夹
base_folder = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"

# 创建主文件夹，如果不存在
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# 设置 Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无界面模式
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 访问原始页面
driver.get("https://www.etit.kit.edu/vertiefungsrichtungen_master.php")

# 获取所有方向页面的链接
soup = BeautifulSoup(driver.page_source, "html.parser")
links = soup.find_all("a", href=True)
direction_links = [link['href'] for link in links if "vertiefungsrichtung" in link['href']]

# 遍历每个方向页面
for idx, direction_link in enumerate(direction_links, 1):  # 这里idx用于生成专业文件夹的名字
    full_direction_url = "https://www.etit.kit.edu/" + direction_link
    print(f"正在访问: {full_direction_url}")
    
    driver.get(full_direction_url)
    direction_soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 使用枚举值来生成文件夹名称，命名为 "vertiefungsrichtungenX"
    direction_name = f"vertiefungsrichtungen{idx}"

    # 将方向名称作为文件夹名，确保路径合法
    direction_folder = os.path.join(base_folder, direction_name)

    # 创建方向文件夹
    if not os.path.exists(direction_folder):
        os.makedirs(direction_folder)

    # 获取 PDF 文件链接
    pdf_links = direction_soup.find_all("a", href=True)
    pdf_urls = [link['href'] for link in pdf_links if link['href'].endswith(".pdf")]

    # 下载 PDF 文件
    for pdf_url in pdf_urls:
        full_pdf_url = "https://www.etit.kit.edu/" + pdf_url
        pdf_name = pdf_url.split("/")[-1]
        
        # 对 PDF 文件名中的特殊字符进行处理
        pdf_name = pdf_name.replace(" ", "_").replace("%20", "_")
        
        pdf_path = os.path.join(direction_folder, pdf_name)
        
        print(f"正在下载: {full_pdf_url}")
        pdf_response = requests.get(full_pdf_url)
        
        # 确保文件夹存在
        os.makedirs(direction_folder, exist_ok=True)
        
        # 保存 PDF 文件
        with open(pdf_path, "wb") as file:
            file.write(pdf_response.content)


# 关闭 Selenium 浏览器并清理资源
driver.quit()

# 清理 WebDriver Manager 缓存文件夹
shutil.rmtree(os.path.expanduser("~/.wdm"), ignore_errors=True)

# 确保 Python 内存清理
driver = None

print("所有PDF文件下载完成！")



正在访问: https://www.etit.kit.edu//vertiefungsrichtungen_master.php
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20WS%202024-25%20-%20Stand%2010.09.2024.pdf
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20ETIT%20SoSe%202025_Stand%2013.02.2025_Erg%c3%a4nzung%20SKOM%20ETIT-MEDT.pdf
正在访问: https://www.etit.kit.edu//vertiefungsrichtungen_master.php
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20WS%202024-25%20-%20Stand%2010.09.2024.pdf
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20ETIT%20SoSe%202025_Stand%2013.02.2025_Erg%c3%a4nzung%20SKOM%20ETIT-MEDT.pdf
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_2.php
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20WS%202024-25%20-%20Stand%2010.09.2024.pdf
正在下载: https://www.etit.kit.edu//rd_download/Terminliste%20ETIT%20SoSe%202025_Stand%2013.02.2025_Erg%c3%a4nzung%20SKOM%20ETIT-MEDT.pdf
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_3.php
正在下载: https://www.etit.kit.edu//rd_download/Terminli

In [4]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 设置下载文件的主文件夹路径
base_folder = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"

# 如果根文件夹不存在，则创建它
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# 设置 Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无界面模式
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 访问原始页面
driver.get("https://www.etit.kit.edu/vertiefungsrichtungen_master.php")

# 获取方向页面的链接
soup = BeautifulSoup(driver.page_source, "html.parser")
links = soup.find_all("a", href=True)
direction_links = [link['href'] for link in links if "vertiefungsrichtung" in link['href']]

# 遍历每个方向页面
for direction_link in direction_links:
    # 提取链接中的数字，用作文件夹名称
    direction_number = direction_link.split('_')[-1].split('.')[0]  # 获取链接中的数字部分

    full_direction_url = "https://www.etit.kit.edu/" + direction_link
    print(f"正在访问: {full_direction_url}")
    
    driver.get(full_direction_url)
    direction_soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 将方向名称作为文件夹名，命名为 "vertiefungsrichtung_X"
    direction_name = f"vertiefungsrichtung_{direction_number}"

    # 将方向名称作为文件夹名，确保路径合法
    direction_folder = os.path.join(base_folder, direction_name)

    # 创建方向文件夹
    if not os.path.exists(direction_folder):
        os.makedirs(direction_folder)

    # 获取 PDF 文件链接
    pdf_links = direction_soup.find_all("a", href=True)
    pdf_urls = [link['href'] for link in pdf_links if link['href'].startswith("https://www.etit.kit.edu/rd_download/") and link['href'].endswith(".pdf")]

    # 下载 PDF 文件
    for pdf_url in pdf_urls:
        pdf_name = pdf_url.split("/")[-1]
        
        # 对 PDF 文件名中的特殊字符进行处理
        pdf_name = pdf_name.replace(" ", "_").replace("%20", "_")
        
        pdf_path = os.path.join(direction_folder, pdf_name)
        
        print(f"正在下载: {pdf_url}")
        pdf_response = requests.get(pdf_url)
        
        # 确保文件夹存在
        os.makedirs(direction_folder, exist_ok=True)
        
        # 保存 PDF 文件
        with open(pdf_path, "wb") as file:
            file.write(pdf_response.content)

# 关闭 Selenium 浏览器
driver.quit()
# 清理 WebDriver Manager 缓存文件夹
shutil.rmtree(os.path.expanduser("~/.wdm"), ignore_errors=True)

# 确保 Python 内存清理
driver = None

print("所有PDF文件下载完成！")


正在访问: https://www.etit.kit.edu//vertiefungsrichtungen_master.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtungen_master.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_2.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_3.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_4.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_5.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_6.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_7.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_8.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_9.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_10.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_11.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_12.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_13.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_14.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_15.php
正在访问: https://www.etit.kit.edu//vertiefungsrichtung_

In [17]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import requests

# 配置路径
base_url = "https://www.etit.kit.edu/vertiefungsrichtungen_master.php"
download_folder = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"  # 更改为你的下载路径

# 初始化 WebDriver
driver = webdriver.Chrome()

# 访问原始页面
driver.get(base_url)

# 等待页面加载并获取所有方向页面链接
WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, 'vertiefungsrichtung_')]")))
direction_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'vertiefungsrichtung_')]")

# 定义正则表达式，用于过滤符合特定格式的页面链接
valid_php_pattern = r"\d{4}\.php$"  # 只匹配以四个数字结尾的php页面


# 遍历每个方向页面
for direction_link in direction_links:
    href = direction_link.get_attribute("href")
    
    # 提取方向编号用于文件夹命名
    direction_number = href.split('_')[-1].split('.')[0]
    direction_name = f"vertiefungsrichtung_{direction_number}"

    # 创建文件夹
    direction_folder = os.path.join(download_folder, direction_name)
    os.makedirs(direction_folder, exist_ok=True)

    # 打开方向页面
    driver.get(href)

    # 等待并获取指向具体页面的链接（可能是2439.php, 2440.php等）
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.php')]")))
    specific_page_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.php')]")

    # 遍历每个具体页面链接（例如：2439.php, 2440.php等）
    for specific_page_link in specific_page_links:
        specific_page_href = specific_page_link.get_attribute("href")

        # 判断是否是指向主页或不需要的页面（例如：回到主页的链接）
        if "home" in specific_page_href or "other" in specific_page_href:  # 根据实际情况调整判断条件
            print(f"跳过不需要的页面: {specific_page_href}")
            continue  # 跳过这个页面
        # 使用正则表达式检查页面是否符合 4 个数字结尾的php页面
        if re.search(valid_php_pattern, specific_page_href):  # 只匹配以 4 个数字结尾的php页面
            print(f"正在处理页面: {specific_page_href}")
        
            # 打开具体页面（例如：2439.php）
            driver.get(specific_page_href)

            # 等待并获取PDF下载链接
            # WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//a[starts-with(@href, 'https://www.etit.kit.edu/rd_download/') and contains(@href, '.pdf')]")))
            pdf_links = driver.find_elements(By.XPATH, "//a[starts-with(@href, 'https://www.etit.kit.edu/rd_download/') and contains(@href, '.pdf')]")
            # 如果没有 PDF 链接，则跳过当前页面
            if not pdf_links:
                print(f"页面 {specific_page_href} 没有 PDF 文件，跳过...")
                continue
            # 下载每个 PDF 文件
            for pdf_link in pdf_links:
                pdf_url = pdf_link.get_attribute("href")
                pdf_name = pdf_url.split("/")[-1]
                pdf_path = os.path.join(direction_folder, pdf_name)

                # 下载 PDF 文件
                pdf_response = requests.get(pdf_url)
                with open(pdf_path, "wb") as pdf_file:
                    pdf_file.write(pdf_response.content)

    print(f"已下载方向 {direction_name} 的所有 PDF 文件")

# 关闭浏览器
driver.quit()




# 清理 WebDriver Manager 缓存文件夹
shutil.rmtree(os.path.expanduser("~/.wdm"), ignore_errors=True)

# 确保 Python 内存清理
driver = None

print("所有PDF文件下载完成！")


正在处理页面: https://www.etit.kit.edu/3110.php
页面 https://www.etit.kit.edu/3110.php 没有 PDF 文件，跳过...


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E477C6A5+28789]
	(No symbol) [0x00007FF7E46E5B20]
	(No symbol) [0x00007FF7E4578F9A]
	(No symbol) [0x00007FF7E458EDA4]
	(No symbol) [0x00007FF7E458D873]
	(No symbol) [0x00007FF7E45816D1]
	(No symbol) [0x00007FF7E4581831]
	(No symbol) [0x00007FF7E457F402]
	(No symbol) [0x00007FF7E4583AB1]
	(No symbol) [0x00007FF7E4620B42]
	(No symbol) [0x00007FF7E45F732A]
	(No symbol) [0x00007FF7E461F7E3]
	(No symbol) [0x00007FF7E45F7103]
	(No symbol) [0x00007FF7E45BFFC0]
	(No symbol) [0x00007FF7E45C1273]
	GetHandleVerifier [0x00007FF7E4AC1AED+3458237]
	GetHandleVerifier [0x00007FF7E4AD829C+3550316]
	GetHandleVerifier [0x00007FF7E4ACDB9D+3507565]
	GetHandleVerifier [0x00007FF7E4842C6A

什么是 href？

href 是 HTML 里超链接（<a> 标签）元素的属性，代表网页链接。
例如：
html
复制
编辑
<a href="https://example.com/page.php">点击这里</a>
在 Selenium 里，你可以通过 get_attribute("href") 获取 <a> 标签的链接：
python
复制
编辑
specific_page_href = specific_page_link.get_attribute("href")
如果这个 specific_page_link 指向：
html
复制
编辑
<a href="https://www.etit.kit.edu/2439.php">方向 2439</a>
那么 specific_page_href 就是 "https://www.etit.kit.edu/2439.php"。
为什么之前的方法不行？

你原来的 specific_page_links 可能是这样获取的：
python
复制
编辑
specific_page_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.php')]")
问题：当你 driver.get(specific_page_href) 访问新页面后，浏览器的 DOM（网页的 HTML 结构）可能会改变，导致 specific_page_links 指向的原始元素不再有效。
什么是 StaleElementReferenceException？

“Stale” 意思是“过时的”，这个错误表示 你获取的 specific_page_links 不再有效，因为：
页面被刷新了：driver.get(specific_page_href) 之后，网页结构变了，导致 specific_page_links 失效。
网页上的链接是动态生成的：可能 DOM 被 JavaScript 修改了，之前的 specific_page_links 就不能用了。

In [18]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, WebDriverException

# 配置路径
base_url = "https://www.etit.kit.edu/vertiefungsrichtungen_master.php"
download_folder = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"

# 初始化 WebDriver
driver = webdriver.Chrome()
driver.get(base_url)

# 等待页面加载并获取所有方向页面链接
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, 'vertiefungsrichtung_')]")))
direction_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'vertiefungsrichtung_')]")

valid_php_pattern = r"\d{4}\.php$"

for direction_link in direction_links:
    href = direction_link.get_attribute("href")
    direction_number = href.split('_')[-1].split('.')[0]
    direction_name = f"vertiefungsrichtung_{direction_number}"
    direction_folder = os.path.join(download_folder, direction_name)
    os.makedirs(direction_folder, exist_ok=True)

    driver.get(href)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.php')]")))
    except TimeoutException:
        print(f"页面 {href} 没有找到 PHP 详情页面链接，跳过...")
        continue

    specific_page_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.php')]")

    for i in range(len(specific_page_links)):  # 遍历索引，避免 stale element
        try:
            specific_page_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.php')]")  # 重新获取元素
            specific_page_href = specific_page_links[i].get_attribute("href")

            if re.search(valid_php_pattern, specific_page_href):
                print(f"处理页面: {specific_page_href}")
                driver.get(specific_page_href)

                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located(
                        (By.XPATH, "//a[starts-with(@href, 'https://www.etit.kit.edu/rd_download/') and contains(@href, '.pdf')]")))
                    pdf_links = driver.find_elements(By.XPATH, "//a[starts-with(@href, 'https://www.etit.kit.edu/rd_download/') and contains(@href, '.pdf')]")

                    if not pdf_links:
                        print(f"页面 {specific_page_href} 没有 PDF 文件，跳过...")
                        continue

                    for pdf_link in pdf_links:
                        pdf_url = pdf_link.get_attribute("href")
                        pdf_name = pdf_url.split("/")[-1]
                        pdf_path = os.path.join(direction_folder, pdf_name)

                        pdf_response = requests.get(pdf_url)
                        with open(pdf_path, "wb") as pdf_file:
                            pdf_file.write(pdf_response.content)

                except TimeoutException:
                    print(f"页面 {specific_page_href} 没有 PDF 文件，跳过...")

        except (StaleElementReferenceException, IndexError):
            print(f"跳过失效元素，重新加载页面 {href} 并获取新的链接")
            driver.get(href)  # 重新加载页面
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.php')]")))
            continue

print("所有 PDF 下载完成")
driver.quit()


处理页面: https://www.etit.kit.edu/3110.php
页面 https://www.etit.kit.edu/3110.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/3435.php
页面 https://www.etit.kit.edu/3435.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/2710.php
页面 https://www.etit.kit.edu/2710.php 没有 PDF 文件，跳过...
跳过失效元素，重新加载页面 https://www.etit.kit.edu/vertiefungsrichtung_2.php 并获取新的链接
处理页面: https://www.etit.kit.edu/3805.php
页面 https://www.etit.kit.edu/3805.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/3805.php
页面 https://www.etit.kit.edu/3805.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/3785.php
页面 https://www.etit.kit.edu/3785.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/4150.php
页面 https://www.etit.kit.edu/4150.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/4150.php
页面 https://www.etit.kit.edu/4150.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/4183.php
页面 https://www.etit.kit.edu/4183.php 没有 PDF 文件，跳过...
处理页面: https://www.etit.kit.edu/4185.php
页面 https://www.etit.kit.edu/4185.php 没有 PDF 文件，跳过.

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E477C6A5+28789]
	(No symbol) [0x00007FF7E46E5B20]
	(No symbol) [0x00007FF7E4578F9A]
	(No symbol) [0x00007FF7E458EDA4]
	(No symbol) [0x00007FF7E458D873]
	(No symbol) [0x00007FF7E45816D1]
	(No symbol) [0x00007FF7E4581831]
	(No symbol) [0x00007FF7E457F402]
	(No symbol) [0x00007FF7E4583AB1]
	(No symbol) [0x00007FF7E4620B42]
	(No symbol) [0x00007FF7E45F732A]
	(No symbol) [0x00007FF7E461F7E3]
	(No symbol) [0x00007FF7E45F7103]
	(No symbol) [0x00007FF7E45BFFC0]
	(No symbol) [0x00007FF7E45C1273]
	GetHandleVerifier [0x00007FF7E4AC1AED+3458237]
	GetHandleVerifier [0x00007FF7E4AD829C+3550316]
	GetHandleVerifier [0x00007FF7E4ACDB9D+3507565]
	GetHandleVerifier [0x00007FF7E4842C6A

改进方案
1. 处理重定向
当访问某个 URL 时，检查它是否会直接重定向到 PDF 文件，而不是只检查 HTML 页面中的 <a> 标签。可以使用 requests 进行初步探测：

python
复制
编辑
import requests

def is_pdf_link(url):
    try:
        response = requests.head(url, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '').lower()
        return 'pdf' in content_type
    except requests.RequestException:
        return False
在 driver.get(url) 之前，先用 is_pdf_link(url) 过滤一下，避免加载不必要的页面。

2. 重新获取 DOM 元素，避免失效
在 driver.get(specific_page_href) 后，不能再使用 specific_page_links 变量，需要重新获取页面的 PDF 链接：

python
复制
编辑
# 访问新页面后，重新获取 DOM
driver.get(specific_page_href)
pdf_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.pdf')]")
for pdf in pdf_links:
    pdf_url = pdf.get_attribute("href")
    print(f"发现 PDF: {pdf_url}")
这样可以确保 pdf_links 是新页面的最新 DOM 结构。

3. 限制页面遍历
如果你的 specific_page_href 变量来自 driver.find_elements(By.TAG_NAME, "a")，那么会遍历所有 <a> 标签，包括无关页面。
可以先筛选出可能包含 PDF 的页面：

python
复制
编辑
valid_pages = [a.get_attribute("href") for a in driver.find_elements(By.TAG_NAME, "a")
               if "studienplan" in a.get_attribute("href").lower()]  # 仅关注特定关键字的页面
这样可以避免访问太多无关页面。

总结
先用 requests.head(url) 预判是否是 PDF 直接下载链接，避免不必要的 Selenium 操作。
访问新页面后，重新获取 DOM 里的 PDF 链接，避免 specific_page_links 失效。
限制 specific_page_href 的来源，确保只遍历可能有用的页面。
你可以尝试改进代码后再运行，看是否能准确获取 PDF。

In [19]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 配置 Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 下载目录
download_dir = "E:/OneDrive - MSFT/.master_data/KIT/SPO2018专业方向"
os.makedirs(download_dir, exist_ok=True)

# 目标起始页面
start_url = "https://www.etit.kit.edu/vertiefungsrichtung_2.php"
driver.get(start_url)
time.sleep(2)  # 等待页面加载

def is_pdf_link(url):
    """判断 URL 是否指向 PDF 文件"""
    try:
        response = requests.head(url, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '').lower()
        return 'pdf' in content_type
    except requests.RequestException:
        return False

# 获取所有可能包含 PDF 的链接
def get_valid_page_links():
    """获取所有可能包含 PDF 的页面链接"""
    links = driver.find_elements(By.TAG_NAME, "a")
    valid_links = []
    for link in links:
        href = link.get_attribute("href")
        if href and "studienplan" in href.lower():  # 只选特定关键字的页面
            valid_links.append(href)
    return valid_links

valid_page_links = get_valid_page_links()

for page_url in valid_page_links:
    driver.get(page_url)
    time.sleep(2)  # 等待加载
    
    # 重新获取页面的 PDF 连接
    pdf_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.pdf')]")
    for pdf in pdf_links:
        pdf_url = pdf.get_attribute("href")
        if is_pdf_link(pdf_url):
            pdf_name = os.path.basename(pdf_url)
            pdf_path = os.path.join(download_dir, pdf_name)
            print(f"下载 PDF: {pdf_url} -> {pdf_path}")
            with open(pdf_path, 'wb') as f:
                f.write(requests.get(pdf_url).content)

driver.quit()

下载 PDF: https://www.etit.kit.edu/rd_download/Terminliste%20WS%202024-25%20-%20Stand%2010.09.2024.pdf -> E:/OneDrive - MSFT/.master_data/KIT/SPO2018专业方向\Terminliste%20WS%202024-25%20-%20Stand%2010.09.2024.pdf
下载 PDF: https://www.etit.kit.edu/rd_download/Terminliste%20ETIT%20SoSe%202025_Stand%2013.02.2025_Erg%c3%a4nzung%20SKOM%20ETIT-MEDT.pdf -> E:/OneDrive - MSFT/.master_data/KIT/SPO2018专业方向\Terminliste%20ETIT%20SoSe%202025_Stand%2013.02.2025_Erg%c3%a4nzung%20SKOM%20ETIT-MEDT.pdf
下载 PDF: https://www.etit.kit.edu/rd_download/Studienpl%c3%a4ne/Studienplan_Bachelor_ETIT_WiSe2023_24.pdf -> E:/OneDrive - MSFT/.master_data/KIT/SPO2018专业方向\Studienplan_Bachelor_ETIT_WiSe2023_24.pdf
下载 PDF: https://www.etit.kit.edu/rd_download/Vertiefungsrichtungen_Bachelor_ETIT_WiSe2025_26_update24.pdf -> E:/OneDrive - MSFT/.master_data/KIT/SPO2018专业方向\Vertiefungsrichtungen_Bachelor_ETIT_WiSe2025_26_update24.pdf
下载 PDF: https://www.etit.kit.edu/rd_download/Ab_11_2019_Bachelor%20SPO%202018%20fuer%20ModHB_SS21.pd

In [20]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import os

# 配置
base_url = "https://www.etit.kit.edu/"
start_url = urljoin(base_url, "vertiefungsrichtungen_master.php")
pdf_root = "KIT_ETIT_PDFs"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 创建保存目录
os.makedirs(pdf_root, exist_ok=True)

def get_direction_links():
    """获取所有专业方向页面链接"""
    print("正在获取专业方向列表...")
    try:
        response = requests.get(start_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 查找所有符合格式的方向链接
        direction_links = []
        for link in soup.find_all('a', href=re.compile(r'vertiefungsrichtung_\d+\.php')):
            full_url = urljoin(base_url, link['href'])
            direction_links.append(full_url)
        return list(set(direction_links))  # 去重
    except Exception as e:
        print(f"获取方向列表失败: {str(e)}")
        return []

def process_direction(direction_url):
    """处理单个专业方向"""
    try:
        print(f"\n处理方向页面: {direction_url}")
        
        # 获取方向页面
        response = requests.get(direction_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 获取方向名称（从h1标题）
        direction_name = "Unnamed_Direction"
        if soup.find('h1'):
            direction_name = soup.find('h1').get_text().strip()
            direction_name = re.sub(r'[\W_]+', '_', direction_name)  # 清理特殊字符
        
        # 查找中间页面链接（包含'Studienpläne'文本的链接）
        middle_page_link = None
        for link in soup.find_all('a', href=True):
            if 'studienpl' in link.get_text().lower():
                middle_page_link = urljoin(base_url, link['href'])
                break
        
        if not middle_page_link:
            print("未找到中间页面链接")
            return

        # 获取中间页面
        print(f"正在解析中间页面: {middle_page_link}")
        response = requests.get(middle_page_link, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 提取所有PDF链接
        pdf_links = []
        for link in soup.find_all('a', href=re.compile(r'\.pdf$', re.I)):
            pdf_url = urljoin(base_url, link['href'])
            pdf_links.append(pdf_url)
        
        # 创建专业方向目录
        dir_path = os.path.join(pdf_root, direction_name)
        os.makedirs(dir_path, exist_ok=True)
        
        # 下载PDF文件
        for pdf_url in pdf_links:
            filename = os.path.basename(requests.utils.unquote(pdf_url))
            filepath = os.path.join(dir_path, filename)
            
            if os.path.exists(filepath):
                print(f"文件已存在: {filename}")
                continue
                
            print(f"正在下载: {filename}")
            try:
                response = requests.get(pdf_url, headers=headers)
                response.raise_for_status()
                with open(filepath, 'wb') as f:
                    f.write(response.content)
            except Exception as e:
                print(f"下载失败: {str(e)}")

    except Exception as e:
        print(f"处理方向时出错: {str(e)}")

if __name__ == "__main__":
    direction_links = get_direction_links()
    print(f"找到 {len(direction_links)} 个专业方向")
    
    for idx, direction_url in enumerate(direction_links, 1):
        print(f"\n[{idx}/{len(direction_links)}]")
        process_direction(direction_url)
    
    print("\n所有任务已完成！下载的文件保存在", os.path.abspath(pdf_root))

正在获取专业方向列表...
找到 23 个专业方向

[1/23]

处理方向页面: https://www.etit.kit.edu/vertiefungsrichtung_17.php
正在解析中间页面: https://www.etit.kit.edu/empfohlener_studienplan_bachelor_etit.php
正在下载: Terminliste WS 2024-25 - Stand 10.09.2024.pdf
正在下载: Terminliste ETIT SoSe 2025_Stand 13.02.2025_Ergänzung SKOM ETIT-MEDT.pdf
正在下载: Studienplan_Bachelor_ETIT_WiSe2023_24.pdf
正在下载: Vertiefungsrichtungen_Bachelor_ETIT_WiSe2025_26_update24.pdf
正在下载: Ab_11_2019_Bachelor SPO 2018 fuer ModHB_SS21.pdf
正在下载: Bis_11_2019_Bachelor SPO 2018_korrigiert.pdf
正在下载: 2021-02-25-Aenderungen ETIT_SPO_ 2015.pdf
正在下载: Änderungen BSc ETIT SPO 2018.pdf
正在下载: Studienplan Bachelor ab WS15_16_Modhb_SS2020.pdf
文件已存在: 2021-02-25-Aenderungen ETIT_SPO_ 2015.pdf

[2/23]

处理方向页面: https://www.etit.kit.edu/vertiefungsrichtung_19.php
正在解析中间页面: https://www.etit.kit.edu/empfohlener_studienplan_bachelor_etit.php
正在下载: Terminliste WS 2024-25 - Stand 10.09.2024.pdf
正在下载: Terminliste ETIT SoSe 2025_Stand 13.02.2025_Ergänzung SKOM ETIT-MEDT.pdf
正在下载: St

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔防止被封

def safe_dir_name(name):
    """生成安全的目录名称"""
    return re.sub(r'[\\/*?:"<>|]', "", name).strip()

def get_direction_info(url):
    """从方向页面获取编号和名称"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 提取编号
        dir_number = re.search(r'vertiefungsrichtung_(\d+)', url).group(1)
        
        # 提取名称
        h1_tag = soup.find('h1')
        dir_name = h1_tag.get_text().split('(')[0].strip() if h1_tag else f"Unnamed_{dir_number}"
        
        return {
            "number": dir_number,
            "name": safe_dir_name(dir_name),
            "url": url
        }
    except Exception as e:
        print(f"获取方向信息失败: {str(e)}")
        return None

def find_middle_page(url):
    """查找包含课程计划的中间页面"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 查找包含关键词的链接
        keywords = ['studienpl', 'curriculum', 'modul']
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text().lower()
            if any(kw in link_text for kw in keywords):
                return urljoin(BASE_URL, a_tag['href'])
        return None
    except Exception as e:
        print(f"查找中间页面失败: {str(e)}")
        return None

def extract_pdf_links(middle_page_url):
    """从中间页面提取真实PDF链接"""
    try:
        response = requests.get(middle_page_url, headers=HEADERS)
        response.raise_for_status()
        
        # 处理可能的重定向
        final_url = response.url
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pdf_links = []
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if href.lower().endswith('.pdf'):
                # 处理双重编码问题
                decoded_url = unquote(unquote(href))
                full_url = urljoin(final_url, decoded_url)
                pdf_links.append(full_url)
        return list(set(pdf_links))
    except Exception as e:
        print(f"提取PDF链接失败: {str(e)}")
        return []

def download_pdfs(pdf_links, dir_path):
    """下载PDF文件到指定目录"""
    os.makedirs(dir_path, exist_ok=True)
    
    for pdf_url in pdf_links:
        try:
            # 获取干净的文件名
            file_name = unquote(pdf_url.split('/')[-1])
            file_path = os.path.join(dir_path, file_name)
            
            if os.path.exists(file_path):
                print(f"✓ 已存在: {file_name}")
                continue
                
            print(f"↓ 正在下载: {file_name}")
            response = requests.get(pdf_url, headers=HEADERS)
            response.raise_for_status()
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            
            print(f"✓ 下载完成: {file_name}")
            time.sleep(REQUEST_DELAY)
            
        except Exception as e:
            print(f"× 下载失败 {pdf_url}: {str(e)}")

def main():
    # 获取所有方向链接
    print("正在解析主页面...")
    main_page = requests.get(START_URL, headers=HEADERS)
    soup = BeautifulSoup(main_page.text, 'html.parser')
    direction_links = [
        urljoin(BASE_URL, a['href']) 
        for a in soup.find_all('a', href=re.compile(r'vertiefungsrichtung_\d+\.php'))
    ]
    
    print(f"发现 {len(direction_links)} 个专业方向")
    
    for idx, direction_url in enumerate(direction_links, 1):
        print(f"\n【{idx}/{len(direction_links)}】处理 {direction_url}")
        
        # 获取方向信息  这已经在每个专业的界面了
        dir_info = get_direction_info(direction_url)
        if not dir_info:
            continue
            
        # 创建目录
        dir_name = f"vertiefungsrichtung_{dir_info['number']}_{dir_info['name']}"
        dir_path = os.path.join(PDF_ROOT, dir_name)
        
        # 查找中间页面
        middle_page = find_middle_page(direction_url)
        if not middle_page:
            print("未找到课程计划页面")
            continue
            
        print(f"发现课程计划页面: {middle_page}")
        
        # 提取PDF链接
        pdf_links = extract_pdf_links(middle_page)
        if not pdf_links:
            print("未找到PDF文件")
            continue
            
        print(f"发现 {len(pdf_links)} 个PDF文件")
        
        # 下载文件
        download_pdfs(pdf_links, dir_path)
    
    print("\n全部下载任务完成！文件保存在:", PDF_ROOT)

if __name__ == "__main__":
    main()

正在解析主页面...
发现 69 个专业方向

【1/69】处理 https://www.etit.kit.edu/vertiefungsrichtung_2.php
发现课程计划页面: https://www.etit.kit.edu/empfohlener_studienplan_bachelor_etit.php
发现 9 个PDF文件
↓ 正在下载: Terminliste WS 2024-25 - Stand 10.09.2024.pdf
✓ 下载完成: Terminliste WS 2024-25 - Stand 10.09.2024.pdf
↓ 正在下载: Studienplan_Bachelor_ETIT_WiSe2023_24.pdf
✓ 下载完成: Studienplan_Bachelor_ETIT_WiSe2023_24.pdf
↓ 正在下载: Terminliste ETIT SoSe 2025_Stand 13.02.2025_Ergänzung SKOM ETIT-MEDT.pdf
✓ 下载完成: Terminliste ETIT SoSe 2025_Stand 13.02.2025_Ergänzung SKOM ETIT-MEDT.pdf
↓ 正在下载: 2021-02-25-Aenderungen ETIT_SPO_ 2015.pdf
✓ 下载完成: 2021-02-25-Aenderungen ETIT_SPO_ 2015.pdf
↓ 正在下载: Bis_11_2019_Bachelor SPO 2018_korrigiert.pdf
✓ 下载完成: Bis_11_2019_Bachelor SPO 2018_korrigiert.pdf
↓ 正在下载: Ab_11_2019_Bachelor SPO 2018 fuer ModHB_SS21.pdf
✓ 下载完成: Ab_11_2019_Bachelor SPO 2018 fuer ModHB_SS21.pdf
↓ 正在下载: Studienplan Bachelor ab WS15_16_Modhb_SS2020.pdf
✓ 下载完成: Studienplan Bachelor ab WS15_16_Modhb_SS2020.pdf
↓ 正在下载: Vertiefungsrich

KeyboardInterrupt: 

好的，我现在需要仔细分析用户的最新反馈，并针对问题进行调整。用户指出之前的代码存在下载错误，尤其是没有准确抓取所需的三个特定PDF链接。用户强调每个方向页面（例如vertiefungsrichtung_2.php）中需要下载的PDF有三个特定标题下的链接：

1. Exemplarischer Studienplan (Download)

2. Individueller Studienplan (Download, Studienbeginn ab WS 2018/19)

3. Empfohlene Wahlmodule

并且每个对应的链接点进去会跳转到实际的PDF地址，例如https://www.etit.kit.edu/3017.php跳转为https://www.etit.kit.edu/rd_download/SGS/ETIT/empfohlene%20Wahlmodule/VR02_empfohlene_Wahlmodule.pdf。用户希望代码能精准抓取这三个链接的PDF，不下载其他无关的PDF。

首先，我需要明确之前的代码为什么会出现错误。根据用户描述，之前的版本可能没有正确识别这三个特定标题下的链接，而是抓取了所有PDF链接，导致下载了不需要的文件。可能的问题包括：

1. **选择器不够精准**：之前可能仅通过关键词匹配（如'studienpl'）来寻找中间页面或PDF链接，但未准确匹配到这三个具体的标题。

2. **页面结构变化**：每个方向页面的结构可能不同，之前的代码未能正确解析到目标链接的位置。

3. **动态跳转处理不足**：部分链接可能经过重定向或URL编码处理，之前的代码未能正确追踪最终PDF地址。

接下来，我需要根据用户的最新需求，重新设计代码逻辑，确保只下载这三个特定标题下的PDF。以下是解决思路：

**步骤一：从方向页面直接定位三个特定链接**

用户明确每个方向页面有三个特定标题的链接，不需要通过中间页面跳转。因此，应直接在方向页面（如vertiefungsrichtung_2.php）中找到这三个标题对应的链接，并获取其最终的PDF URL。

**步骤二：精确匹配标题文本**

使用BeautifulSoup精确匹配这三个德文标题，确保只抓取目标链接。需要处理可能的空格、大小写和标点符号差异。

**步骤三：处理链接跳转**

部分链接（如3017.php）可能通过服务器端重定向到实际的PDF地址。需要使用requests库的`allow_redirects=True`来自动跟随重定向，并获取最终的PDF URL。

**步骤四：文件名和目录结构**

根据用户要求，每个方向生成一个文件夹，名称使用vertiefungsrichtung_编号，并将三个PDF分别保存为特定名称，避免文件名混乱。

**步骤五：错误处理和日志输出**

增强错误处理，确保每个步骤都有明确的打印输出，方便用户跟踪进度和排查问题。

**具体实现细节：**

1. **标题匹配**：

- 使用精确的德文标题文本进行匹配，注意可能的变体（如括号内的内容）。例如，使用正则表达式来匹配标题的开头部分。

- 例如，匹配“Exemplarischer Studienplan”时，可以忽略括号内的内容，使用`re.compile(r'^Exemplarischer Studienplan', re.IGNORECASE)`。

2. **链接提取**：

- 在方向页面中找到包含这三个标题的`<a>`标签，提取其href属性。

- 处理相对URL，拼接成完整URL。

3. **处理重定向**：

- 对每个提取的链接发送HEAD请求（节省带宽），跟随重定向，获取最终的PDF URL。

- 使用HEAD请求而不是GET请求，以避免下载整个PDF文件，只获取头部信息确认最终地址。

4. **文件名规范**：

- 根据每个标题生成固定的文件名，例如：

- Exemplarischer Studienplan → exemplarischer_studienplan.pdf

- Individueller Studienplan → individueller_studienplan.pdf

- Empfohlene Wahlmodule → empfohlene_wahlmodule.pdf

- 避免使用原始URL中的动态文件名，确保文件名统一。

5. **目录结构**：

- 每个方向创建独立文件夹，路径为E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_{编号}。

- 确保目录存在，避免因路径问题导致保存失败。

6. **增强日志**：

- 打印每个步骤的状态，如找到的链接数量、下载进度、错误信息等。

- 使用清晰的标识符区分不同阶段的信息，方便用户追踪。

**潜在问题及解决方案：**

- **标题文本变化**：如果网站更新了标题的措辞，正则表达式需要调整。可考虑更灵活的正则表达式，或允许用户自定义标题关键词。

- **重定向链**：某些链接可能经过多次重定向。确保requests自动处理所有重定向，并获取最终URL。

- **编码问题**：PDF URL可能包含特殊字符（如空格编码为%20），需正确解码。使用`urllib.parse.unquote`处理URL编码。

- **网络请求失败**：增加重试机制，设置合理的超时时间，并处理可能的异常。

**代码结构优化：**

- 将功能模块化，分为获取方向链接、处理每个方向、提取PDF链接、下载PDF等函数。

- 使用会话（requests.Session）保持连接，提高效率。

- 添加延迟（time.sleep）以避免触发反爬机制。

通过以上分析，可以重新编写代码，确保精准抓取目标PDF，并满足用户的所有要求。

In [22]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, unquote
import os
import time

# 配置参数
BASE_URL = "https://www.etit.kit.edu/"
START_URL = urljoin(BASE_URL, "vertiefungsrichtungen_master.php")
PDF_ROOT = r"E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "de-DE,de;q=0.9"
}
REQUEST_DELAY = 1  # 请求间隔

# 定义目标PDF标题（德语原文）
TARGET_LINKS = {
    "exemplarischer": {
        "pattern": re.compile(r'^Exemplarischer Studienplan\b', re.I),
        "filename": "Exemplarischer_Studienplan.pdf"
    },
    "individueller": {
        "pattern": re.compile(r'^Individueller Studienplan\b.*ab WS 2018/19', re.I),
        "filename": "Individueller_Studienplan.pdf"
    },
    "empfohlene": {
        "pattern": re.compile(r'^Empfohlene Wahlmodule\b', re.I),
        "filename": "Empfohlene_Wahlmodule.pdf"
    }
}

def create_directory(dir_path):
    """创建存储目录"""
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"创建目录：{dir_path}")
    except Exception as e:
        print(f"目录创建失败：{str(e)}")
        return False
    return True

def get_final_pdf_url(link_url):
    """获取最终PDF地址（处理重定向）"""
    try:
        with requests.Session() as session:
            response = session.head(link_url, headers=HEADERS, allow_redirects=True, timeout=10)
            response.raise_for_status()
            return response.url
    except Exception as e:
        print(f"解析PDF地址失败：{str(e)}")
        return None

def process_direction(direction_url, dir_number):
    """处理单个专业方向"""
    print(f"\n{'='*40}\n正在处理方向：{dir_number}")
    
    try:
        # 创建存储目录
        dir_path = os.path.join(PDF_ROOT, f"vertiefungsrichtung_{dir_number}")
        if not create_directory(dir_path):
            return

        # 获取方向页面
        response = requests.get(direction_url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 查找所有包含PDF的链接
        pdf_links = {}
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text(strip=True)
            for key, config in TARGET_LINKS.items():
                if config["pattern"].search(link_text):
                    # 获取原始链接
                    raw_link = urljoin(direction_url, a_tag['href'])
                    # 获取最终PDF地址
                    pdf_url = get_final_pdf_url(raw_link)
                    if pdf_url and pdf_url.lower().endswith('.pdf'):
                        pdf_links[key] = {
                            "url": pdf_url,
                            "filename": config["filename"]
                        }
                    break

        # 下载所有找到的PDF
        download_count = 0
        for key, data in pdf_links.items():
            file_path = os.path.join(dir_path, data["filename"])
            if os.path.exists(file_path):
                print(f"文件已存在：{data['filename']}")
                continue

            print(f"正在下载：{data['filename']}")
            try:
                response = requests.get(data["url"], headers=HEADERS)
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                download_count += 1
                time.sleep(REQUEST_DELAY)
            except Exception as e:
                print(f"下载失败：{str(e)}")

        print(f"完成下载：{download_count}/3 个文件")

    except Exception as e:
        print(f"处理方向时发生错误：{str(e)}")

def main():
    print("正在解析主页面...")
    try:
        # 获取所有方向链接
        response = requests.get(START_URL, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 匹配所有vertiefungsrichtung_数字.php的链接
        direction_links = []
        pattern = re.compile(r'vertiefungsrichtung_(\d+)\.php')
        for a_tag in soup.find_all('a', href=pattern):
            full_url = urljoin(BASE_URL, a_tag['href'])
            match = pattern.search(full_url)
            if match:
                direction_links.append( (full_url, match.group(1)) )  # (URL, 编号)

        total = len(direction_links)
        print(f"找到 {total} 个专业方向")

        # 遍历处理每个方向
        for idx, (url, number) in enumerate(direction_links, 1):
            print(f"\n▶ 进度：{idx}/{total}")
            process_direction(url, number)
        
        print("\n✅ 所有方向处理完成！")

    except Exception as e:
        print(f"主流程错误：{str(e)}")

if __name__ == "__main__":
    main()

正在解析主页面...
找到 69 个专业方向

▶ 进度：1/69

正在处理方向：2
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_2
正在下载：Exemplarischer_Studienplan.pdf
正在下载：Individueller_Studienplan.pdf
正在下载：Empfohlene_Wahlmodule.pdf
完成下载：3/3 个文件

▶ 进度：2/69

正在处理方向：3
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_3
正在下载：Exemplarischer_Studienplan.pdf
正在下载：Individueller_Studienplan.pdf
正在下载：Empfohlene_Wahlmodule.pdf
完成下载：3/3 个文件

▶ 进度：3/69

正在处理方向：4
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_4
正在下载：Exemplarischer_Studienplan.pdf
正在下载：Individueller_Studienplan.pdf
正在下载：Empfohlene_Wahlmodule.pdf
完成下载：3/3 个文件

▶ 进度：4/69

正在处理方向：5
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_5
正在下载：Exemplarischer_Studienplan.pdf
正在下载：Individueller_Studienplan.pdf
正在下载：Empfohlene_Wahlmodule.pdf
完成下载：3/3 个文件

▶ 进度：5/69

正在处理方向：6
创建目录：E:\OneDrive - MSFT\.master_data\KIT\SPO2018专业方向\vertiefungsrichtung_6
正在下载：Exemplarischer_Studienplan.pdf
正